In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "7"  # specify which GPU(s) to be used

In [3]:
from utils import Config, Data_Manager, model_selection
cfg = Config()
dm = Data_Manager(cfg)

model, tokenizer = model_selection(cfg)
sae_dict = dm.load_dict(dict_type='sae')
actmax_dict = dm.load_dict(dict_type='actmax')
device_dict = dm.load_dict(dict_type='device', model=model)
std_dict = dm.load_dict(dict_type='act-std')
for layer in cfg.hook_layers:
    sae_dict[layer] = sae_dict[layer].to(device_dict[layer])

Loading model...


Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [2]:
model_ids = [
    "google/gemma-3-270m-it", 
    "Qwen/Qwen3-0.6B", 
    "meta-llama/Llama-3.2-1B-Instruct", 
    "Qwen/Qwen3-1.7B", 
    "meta-llama/Llama-3.2-3B-Instruct", 
    "google/gemma-3-4b-it",
    "meta-llama/Llama-3.1-8B-Instruct",
    "google/gemma-3-12b-it", 
    "Qwen/Qwen3-14B", 
    "google/gemma-3-27b-it",
    "Qwen/Qwen3-32B",
    "meta-llama/Llama-3.3-70B-Instruct",
]

In [22]:
import json
from utils import Config, Data_Manager, model_selection
for model_id in model_ids:
    cfg = Config(model_id)
    dm = Data_Manager(cfg)
    
    with open(f'{cfg.itv_str_dir}/itv_str_dict_v{cfg.current_v}.json', 'r') as f:
        itv_str_dict = json.load(f)    
    
    path = f'./data/{model_id}/itv_strength_v4/itv_str_dict_v4.1.json'
    # if the folder does not exist, create it
    os.makedirs(os.path.dirname(path), exist_ok=True)

    # save json in a structured way
    with open(path, 'w') as f:
        json.dump(itv_str_dict, f, indent=4)
        


In [ ]:
import json
from utils import Config, Data_Manager, model_selection
for model_id in model_ids:
    cfg = Config(model_id)
    dm = Data_Manager(cfg)
    
    
    path = f'./data/{model_id}/result_summary.json'
    # create empty json file
    with open(path, 'w') as f:
        json.dump({}, f, indent=4)
    

In [13]:
import os
import zipfile
import pandas as pd
from utils import Config, Data_Manager, model_selection
for model_id in model_ids:
    cfg = Config(model_id)
    dm = Data_Manager(cfg)
    out_df = dm.load_output('activation_spread_eval')
    out_df['output_text'] = ''
    
    new_path = f'./data/{model_id}/spread_eval_v4/'
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    # out_df.to_csv(f'{new_path}/outcome_v4.1.csv', index=False)
    
    # save zip file
    # zip the csv file
    with zipfile.ZipFile(f'{new_path}/outcome_v4.1.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
        out_df.to_csv(f'{new_path}/outcome_v4.1.csv', index=False)
        zipf.write(f'{new_path}/outcome_v4.1.csv', arcname='outcome_v4.1.csv')
        os.remove(f'{new_path}/outcome_v4.1.csv')


In [12]:
with zipfile.ZipFile(f'{new_path}/outcome_v4.1.zip', 'r') as zipf:
    zipf.extractall(new_path)
